<a href="https://colab.research.google.com/github/luckhazzard03/-IBM-AI-Developer/blob/main/DatosSqulite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python3 --version

Python 3.11.11


In [2]:
import pandas as pd
pd.__version__

'2.2.2'

In [3]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 68.7 MB/s eta 0:00:00


In [5]:
#Se adjunta archivos pdf desde pc
from google.colab import files
uploaded = files.upload()

Saving documento_ley_comercio.pdf to documento_ley_comercio.pdf
Saving documento_ley_datos.pdf to documento_ley_datos.pdf
Saving documento_ley_propiedad.pdf to documento_ley_propiedad.pdf
Saving documento_ley_seguridad.pdf to documento_ley_seguridad.pdf
Saving documento_ley_transparencia.pdf to documento_ley_transparencia.pdf


In [6]:
#Verificación de archivos subidos correctamente
import os
print(os.listdir())

['.config', 'documento_ley_propiedad.pdf', 'documento_ley_comercio.pdf', 'documento_ley_datos.pdf', 'documento_ley_transparencia.pdf', 'documento_ley_seguridad.pdf', 'sample_data']


In [8]:
#Instalar SQLAlcheamy
!pip install sqlalchemy

In [9]:
# verificación de la versión SQL
import sqlalchemy
print(sqlalchemy.__version__)  # Debería mostrar 2.0.39

2.0.39


In [14]:
from sqlalchemy import create_engine, Column, Integer, String, Date
from sqlalchemy.orm import declarative_base, sessionmaker

# Crear la conexión a SQLite
engine = create_engine("sqlite:///leyes.db")
Base = declarative_base()

# Definir el modelo de datos
class Ley(Base):
    __tablename__ = "leyes"

    id = Column(Integer, primary_key=True, autoincrement=True)  # SQLite generará el ID automáticamente
    nombre = Column(String, nullable=False)
    fecha = Column(Date, nullable=False)
    tipo_ley = Column(String, nullable=False)

# Crear la base de datos y la tabla
Base.metadata.create_all(engine)

# Crear una sesión para interactuar con la base de datos
Session = sessionmaker(bind=engine)  # Corregido sessionmaker
session = Session()

print("✅ Base de datos y tabla creadas correctamente.")


✅ Base de datos y tabla creadas correctamente.


In [15]:
import fitz  # PyMuPDF
from datetime import datetime

# Lista de PDFs
pdf_files = [
    "documento_ley_comercio.pdf",
    "documento_ley_datos.pdf",
    "documento_ley_propiedad.pdf",
    "documento_ley_seguridad.pdf",
    "documento_ley_transparencia.pdf"
]

# Extraer datos y guardarlos en la BD
for pdf_document in pdf_files:
    doc = fitz.open(pdf_document)

    for page_num in range(len(doc)):
        text = doc[page_num].get_text("text").split("\n")  # Obtener texto línea por línea

        # Buscar la posición del primer dato (evitar encabezados)
        start_index = None
        for i, line in enumerate(text):
            if line.strip().isdigit():  # Si una línea es un número, asumimos que es un ID
                start_index = i
                break

        if start_index is not None:
            for i in range(start_index, len(text), 4):
                try:
                    # Extraer datos
                    nombre = text[i+1].strip()
                    fecha = datetime.strptime(text[i+2].strip(), "%Y-%m-%d").date()
                    tipo_ley = text[i+3].strip()

                    # Insertar en la base de datos SIN ID (SQLite lo genera automáticamente)
                    nueva_ley = Ley(nombre=nombre, fecha=fecha, tipo_ley=tipo_ley)
                    session.add(nueva_ley)
                except (IndexError, ValueError):
                    continue  # Evita errores por datos incompletos

    doc.close()

# Guardar los cambios en la BD
session.commit()
print("✅ Datos guardados en SQLite correctamente.")


✅ Datos guardados en SQLite correctamente.


In [16]:
# Consultar datos de la BD
leyes = session.query(Ley).all()

# Convertir a DataFrame para visualización
df = pd.DataFrame([(l.id, l.nombre, l.fecha.strftime("%d-%m-%Y"), l.tipo_ley) for l in leyes],
                  columns=["ID", "Nombre", "Fecha", "Tipo de Ley"])

print(df)

    ID             Nombre       Fecha                   Tipo de Ley
0    1     Luis Fernández  14-02-2024   Ley de Comercio Electrónico
1    2       Camila Ortiz  09-08-2023   Ley de Comercio Electrónico
2    3         David Ruiz  22-06-2022   Ley de Comercio Electrónico
3    4       Andrea Pérez  30-04-2021   Ley de Comercio Electrónico
4    5       Javier López  15-10-2020   Ley de Comercio Electrónico
5    6         Juan Pérez  21-03-2024    Ley de Protección de Datos
6    7        María López  14-06-2023    Ley de Protección de Datos
7    8        Carlos Díaz  30-08-2022    Ley de Protección de Datos
8    9          Ana Gómez  25-02-2021    Ley de Protección de Datos
9   10      Pedro Ramírez  18-09-2020    Ley de Protección de Datos
10  11       Martín Vélez  12-04-2024  Ley de Propiedad Intelectual
11  12      Gabriela Ríos  20-10-2023  Ley de Propiedad Intelectual
12  13   Sebastián Torres  28-12-2022  Ley de Propiedad Intelectual
13  14     Daniela Suárez  14-06-2021  Ley de Pr

In [17]:
# Consultar datos y exportar a Excel
df.to_excel("leyes.xlsx", index=False)

# Descargar el archivo en Google Colab
from google.colab import files
files.download("leyes.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>